In [51]:
import os
import io
import dotenv
from pprint import pprint
import six
import pathlib
import json

import cv2

# Load env
dotenv.load_dotenv('../.env')

# Imports the Google Cloud client library
from google.cloud import vision
#from google.cloud.vision import types

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

class BusinessCardReader:
    required_entities = {'ORGANIZATION':'','PERSON':'','LOCATION':'','ADDRESS':'','PHONE_NUMBER':'',}
    alpha = 0.3
    
    def __init__(self):
        pass
    
    def detect_text(self,path):
        """Detects text in the file."""
        from google.cloud import vision
        client = vision.ImageAnnotatorClient()
    
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
    
        image = vision.types.Image(content=content)
    
        self.response = client.text_detection(image=image)
        texts = self.response.text_annotations
        print('Texts:')
    
        for text in texts:
            print('\n"{}"'.format(text.description))
    
            vertices = (['({},{})'.format(vertex.x, vertex.y)
                        for vertex in text.bounding_poly.vertices])
    
            print('bounds: {}'.format(','.join(vertices)))

    def detect_text_uri(self,uri):
        """Detects text in the file located in Google Cloud Storage or on the Web.
        """
        #from google.cloud import vision
        client = vision.ImageAnnotatorClient()
        image = vision.types.Image()
        image.source.image_uri = uri
    
        self.response = client.text_detection(image=image)
        texts = self.response.text_annotations
        print('Texts:')
    
        for text in texts:
            print('\n"{}"'.format(text.description))
    
            vertices = (['({},{})'.format(vertex.x, vertex.y)
                        for vertex in text.bounding_poly.vertices])
    
            print('bounds: {}'.format(','.join(vertices)))
        
    
    
    def extract_entities(self,):
        client = language.LanguageServiceClient()
        
        text = self.response.text_annotations[0].description
        
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        
        # Instantiates a plain text document.
        document = types.Document(
            content=text,
            type=enums.Document.Type.PLAIN_TEXT)
        
        # Detects entities in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
        entities = client.analyze_entities(document).entities
        
        for entity in entities:
            entity_type = enums.Entity.Type(entity.type)
            if entity_type.name in self.required_entities:
                #print(entity.name)
                self.required_entities[entity_type.name] += entity.name
            #print('=' * 20)
            #print(u'{:<16}: {}'.format('name', entity.name))
            #print(u'{:<16}: {}'.format('type', entity_type.name))
            #print(u'{:<16}: {}'.format('salience', entity.salience))
            #print(u'{:<16}: {}'.format('wikipedia_url',
            #      entity.metadata.get('wikipedia_url', '-')))
            #print(u'{:<16}: {}'.format('mid', entity.metadata.get('mid', '-')))
        pprint(self.required_entities)
        
    def save_result(self):
        pass
        
        
    def read_business_card(self,path):
        image_path_list = list(pathlib.Path(path).glob('*.jpg'))
        for image_path in image_path_list:
            #print(image_path)
            #print(image_path.parent)
            #print(image_path.parent / 'output')
            #print(image_path.name)
            #print(image_path.suffix)
            #print(str(image_path.name).strip(image_path.suffix))
            
            self.detect_text(str(image_path))
            self.extract_entities()
            
            #save result
            save_path = image_path.parent / 'output'
            save_file_path = save_path / image_path.name
            #print(save_file_path)
            
            #save overlayed image
            image = cv2.imread(str(image_path))
            overlay = image.copy()
            output = image.copy()
            texts = self.response.text_annotations
            for i,text in enumerate(texts):
                #print('\n"{}"'.format(text.description))
                #vertices = (['({},{})'.format(vertex.x, vertex.y)
                #            for vertex in text.bounding_poly.vertices])
                #print('bounds: {}'.format(','.join(vertices)))
                #print(text.bounding_poly.vertices[0],text.bounding_poly.vertices[2])
                if(i == 0):
                    continue
                
                cv2.rectangle(overlay,
                              (text.bounding_poly.vertices[0].x,text.bounding_poly.vertices[0].y),
                              (text.bounding_poly.vertices[2].x,text.bounding_poly.vertices[2].y),
                              (0,255,0),
                              -1)
            cv2.addWeighted(overlay, self.alpha, output, 1 - self.alpha,0, output)
            cv2.imwrite(str(save_file_path),output)
            
            #save raw text
            text_file_name = save_file_path.with_suffix('.txt')
            with io.open(str(text_file_name),'w') as file:
                print(self.response.text_annotations[0].description)
                file.write(self.response.text_annotations[0].description)
            
            #save rabeling result
            json_file_name = save_file_path.with_suffix('.json')
            #print(json_file_name)
            with io.open(str(json_file_name), 'w') as file:
                json.dump(self.required_entities,file)
        

test_url = 'http://web.cs.wpi.edu/~claypool/mmsys-dataset/2011/stanford/mvs_images/business_cards/Reference/016.jpg'
image_path = '../images/001.jpg'
file_path = '../images/business_cards/Reference/'

business_card_reader = BusinessCardReader()
#business_card_reader.detect_text_uri(test_url)
#business_card_reader.extract_entities()
#business_card_reader.read_business_card_uri(test_url)

business_card_reader.read_business_card(file_path)


Texts:

"Institute for Computer Sciences, Social Informatics
and Telecommunications Engineering
ICST.ORG
Gabriella MAGYAR
Conference Coordinator
Begijnhoflaan 93a, B-9000 Gent, Belgium
phone: +32 9 3299 425
e-mail: gabriella.magyar@icst.org
skype: gabriella.magyar-icst
Gent Boston Hong Kong Sydney Alexandria
www
"
bounds: (57,14),(965,14),(965,514),(57,514)

"Institute"
bounds: (277,19),(370,18),(370,42),(277,43)

"for"
bounds: (379,18),(411,18),(411,42),(379,42)

"Computer"
bounds: (420,18),(539,17),(539,47),(420,48)

"Sciences,"
bounds: (548,17),(663,16),(663,45),(548,46)

"Social"
bounds: (674,16),(745,16),(745,41),(674,41)

"Informatics"
bounds: (757,15),(885,14),(885,40),(757,41)

"and"
bounds: (276,59),(319,59),(319,83),(276,83)

"Telecommunications"
bounds: (329,58),(576,58),(576,82),(329,82)

"Engineering"
bounds: (587,57),(728,57),(728,87),(587,87)

"ICST.ORG"
bounds: (57,142),(210,138),(211,174),(58,178)

"Gabriella"
bounds: (404,218),(564,217),(564,253),(404,254)

"MAGYAR"
b

KeyboardInterrupt: 